In [4]:
import requests 
from bs4 import BeautifulSoup 

atm_url = 'https://www.trustpilot.com/categories/atm'

response = requests.get(atm_url)

soup = BeautifulSoup(response.content, "html.parser")

best_atm = soup.find_all('div', attrs = {'class': "paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2"})

In [5]:
import pandas as pd
name = []
trustscore = []
num_review = []
domain = []

for company in best_atm:

    name.append(company.find('p', {'class': 'typography_heading-xs__jSwUz typography_appearance-default__AAY17 styles_displayName__GOhL2'}).text)
    
    try:
        trustscore_text = company.find('span', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_trustScore__8emxJ'}).text
        trustscore_value = trustscore_text.split()[1]
        trustscore.append(trustscore_value)
    except AttributeError:
        trustscore.append(None)
    
    try:
        num_review_text = company.find('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_ratingText__yQ5S7'}).text
        num_review_value = num_review_text.split('|')[1].split()[0]
        num_review.append(num_review_value)
    except AttributeError:
        num_review.append(None)
    
    domain.append(company.find('div', {'class': 'styles_wrapper___E6__ styles_categoriesLabels__FiWQ4 styles_desktop__U5iWw'}).text)

    
atm_df = pd.DataFrame(list(zip(name, trustscore, num_review, domain)), columns=['company_name', 'trustscore', 'total_reviews', 'domain'])



In [6]:
atm_df.insert(0,'company_id', atm_df.index)

for i in range(len(atm_df)):
    text = "11{number}"
    atm_df.company_id[i] = text.format(number = i)

/var/folders/td/46d41brx09b2dvddbff_mvfm0000gn/T/ipykernel_4899/4263009943.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atm_df.company_id[i] = text.format(number = i)


In [7]:
atm_df = atm_df.dropna(subset=['total_reviews', 'trustscore'])

In [8]:
atm_df.to_csv('atm.csv', index=False)
atm_df

,company_id,company_name,trustscore,total_reviews,domain
0,110,Evergreen Credit Union,4.9,222,ATM·Credit Union·Financial Institution
1,111,Liberty First Credit Union,4.8,198,Car Finance and Loan Company·Credit Union·ATM·...
2,112,Crypto Dispensers,2.3,120,Investment Service·Non-Bank Financial Service·...
3,113,Swadesh,4.3,9,ATM·Bank·Financial Institution·Money Transfer ...
4,114,Heritage Valley Federal Credit Union,4.1,11,ATM·Bank·Federal Credit Union·Financial Instit...
5,115,The Payment HQ,3.8,2,Business to Business Service·Payment Service·F...
6,116,ASDFTEST,3.7,1,ATM
7,117,Kryptominerstech,3.7,1,Money Order Service·Business Center·Gold Minin...
8,118,WPC Merchant Services & Credit Card Processing,3.7,1,ATM·Business to Business Service
9,119,Meriwest Credit Union,3.5,8,Credit Counseling Service·Car Finance and Loan...


In [9]:
#Create table best_atm(company_id bigint not null primary key, company_name varchar not null, trustscore float, total_reviews integer, domain varchar not null);

In [10]:
#Establish connection with PostgreSQL using psycopg2

import psycopg2
import numpy as np
import psycopg2.extras as extras

#Function to insert values into existing table
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    col = ','.join(list(df.columns))
    # SQL query to execute
    query = "DELETE FROM %s; INSERT INTO %s(%s) VALUES %%s" % (table, table, col)
    
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()
  
  
conn = psycopg2.connect(
    database="atm_scraping", user='postgres', password='postgres', host='127.0.0.1', port='5432'
)

In [11]:
execute_values(conn, atm_df, 'best_atm')

the dataframe is inserted
